# Introduction

This notebook I created with extra classes, functions that I developed during the competition. About most of these features I read in papers, but not all of them proved effective when added to the best competition solutions/notebooks. But some of them did improve the ensemble of several top solutions (I added some functions to different models). So I just decided to test all of these my preprocessors in one notebook without common best notebook features.

The notebook is created as a utility script, and can be used in main inference notebook by importing its code and executing needed functions. Also it contains dumped model and oof predictions.

# Libraries

In [1]:
print('__name__ is: ', __name__)

__name__ is:  __main__


In [2]:
class CONFIG:
#     solution_name = 'LGBMRegressor'
    solution_name = 'XGBRegressor'
    n_splits = 10
    random_state = 422
    data_path = '/kaggle/input/linking-writing-processes-to-writing-quality/'

In [3]:
import pandas as pd
import numpy as np
import re
from scipy.stats import skew, kurtosis
import warnings

from tqdm import tqdm
from pandas import Index

from scipy.stats import skew, kurtosis, iqr, entropy
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import jensenshannon
from scipy.stats import skew, kurtosis, iqr, entropy, gmean, tmean
from scipy.signal import argrelmin, argrelmax
import joblib

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

if __name__ == '__main__':
    print('__name__ is: ', __name__)
    from lightgbm import LGBMRegressor
    from xgboost import XGBRegressor
    from sklearn.model_selection import StratifiedKFold
    from copy import deepcopy
    from sklearn.ensemble import VotingRegressor   

__name__ is:  __main__


# Feature Engineering

In [4]:
def iqr_nan(x):
    return iqr(x.to_numpy(), nan_policy='omit')

def entropy_nan(x, notna=True):
    if notna:
        return entropy(x[x.notna()].to_numpy().astype('float32'))
    else:
        return entropy(x.to_numpy().astype('float32'))
    
def relextrema_count(x):
    x = x.to_numpy()
    min_count = len(argrelmin(x)[0])
    max_count = len(argrelmax(x)[0])
    return (min_count + max_count) #/(len(x)+1e-7)

def jensenshannon_degree(x):
    a = x[x.notna()].to_numpy()
    if len(a) == 0:
        return np.NaN
    b = np.repeat(1/len(a), len(a))
    return jensenshannon(a, b)

In [5]:
def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText

def get_essay_df(df):
    df = df[df.activity != 'Nonproduction']
    temp = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df

PreprocessorTimeWindows_* classes calculate statistics of events in time windows (ex. 60 sec).

In [6]:
class PreprocessorTimeWindows_Base:
    def __init__(self):
        pass
        
    def get_time_windows(self, df, time_win):
        bins = range(0, df['up_time'].max() + time_win, time_win)
        window_series = pd.cut(df['up_time'], bins, labels=False, right=False)
        return window_series

In [7]:
class PreprocessorTimeWindows_Activities(PreprocessorTimeWindows_Base):
    def __init__(self, agg_methods, time_windows=[30], filter_field=None, filter_values=[], field_suffix=''):
        self.agg_methods = agg_methods
        self.time_windows = time_windows
        self.filter_field = filter_field
        self.filter_values = filter_values
        self.field_suffix = field_suffix
    
    def get_time_window_agg(self, data, win_field='window', agg_method='mean'):
        win_max = data.iloc[-1][win_field]
        new_index = Index(np.arange(win_max+1), name=win_field)
        agg_windows_df = data.groupby(win_field).apply(
            lambda x: x.count()).rename(columns={win_field: 'events_count'}).reindex(new_index).fillna(0)
        agg_result = agg_windows_df['events_count'].agg(agg_method)
        return agg_result
    
    def get_windows_stats(self, df, win_field):
        feats_df = pd.DataFrame({'id': df['id'].unique().tolist()})
        stat_type = 'activity_count'
        pbar = tqdm(self.agg_methods)
        for method in pbar:
            if isinstance(method, str):
                method_name = method
            else:
                method_name = method.__name__
            pbar.set_postfix(method=method_name)
            agg_field_name = f'{win_field}_{stat_type}_{method_name}{self.field_suffix}'
            tmp_df = df.groupby('id')[[win_field]].apply(
                lambda x: self.get_time_window_agg(
                    x, win_field=win_field, agg_method=method
                )).to_frame().rename(columns={0: agg_field_name})
            feats_df = feats_df.merge(tmp_df, on='id', how='left')
        return feats_df                
    
    def make_feats(self, df, fillna=None):
        print("Starting to engineer features")
        filter_col = [self.filter_field] if self.filter_field != None else []
        data_df = df[['id', 'down_time', 'up_time'] + filter_col]
        merged_df = pd.DataFrame({'id': df['id'].unique().tolist()})
        for t_win in self.time_windows:
            win_field = f'window_{t_win}s'
            data_df[win_field] = self.get_time_windows(data_df, time_win=t_win*1000)
            if self.filter_field != None:
                data_df = data_df[data_df[self.filter_field].isin(self.filter_values)]
            if data_df.empty:
                data_df['id'] = df['id'].unique()
                data_df[win_field] = data_df[win_field].fillna(0)
            print('Calculating window stats')
            feats_df = self.get_windows_stats(data_df, win_field) #.reset_index()
            merged_df = merged_df.merge(feats_df, on='id', how='left')
            if fillna != None:
                merged_df = merged_df.fillna(fillna)
        
        return merged_df

In [8]:
class PreprocessorTimeWindows_Pauses(PreprocessorTimeWindows_Base):
    def __init__(self, agg_methods, time_windows=[30]):
        self.agg_methods = agg_methods
        self.time_windows = time_windows
        self.pauses_splits = [
#             ('pause_05_sec', [0.5, 1]),
            ('pause_1_sec', [1, 1.5]),
            ('pause_15_sec', [1.5, 2]),
            ('pause_2_sec', [2, 3]),
#             ('pause_3_sec', [3, np.inf]),
        ]
        
    def get_time_diff(self, df):
        up_time_lagged = df.groupby('id')['up_time'].shift(1).fillna(df['down_time'])
        time_diff = abs(df['down_time'] - up_time_lagged) / 1000
        return time_diff
        
    def get_time_window_agg(self, data, win_field='window', agg_method='mean', lim_lower=1, lim_upper=1.5):
        win_max = data.iloc[-1][win_field]
        new_index = Index(np.arange(win_max+1), name=win_field)
        group_window = data.groupby(win_field)['time_diff']
        agg_windows_df = group_window.apply(
            lambda x: ((x >= lim_lower) & (x < lim_upper)).sum()
        ).reindex(new_index).fillna(0) #.rename(columns={win_field: 'events_count'}).reindex(new_index).fillna(0)
        agg_result = agg_windows_df.agg(agg_method)
        return agg_result
    
    def get_windows_stats(self, df, win_field):
        feats_df = pd.DataFrame({'id': df['id'].unique().tolist()})
        pbar = tqdm(self.pauses_splits)
        for params in pbar:
            pause_name, lim_lower, lim_upper = params[0], params[1][0], params[1][1]
            for method in self.agg_methods:
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                pbar.set_postfix(pause=pause_name, method=method_name)
                agg_field_name = f'{win_field}_{pause_name}_{method_name}'
                tmp_df = df.groupby('id')[['id', win_field, 'time_diff']].apply(
                    lambda x: self.get_time_window_agg(
                        x, win_field=win_field, agg_method=method, lim_lower=lim_lower, lim_upper=lim_upper
                    )).to_frame().reset_index().rename(columns={0: agg_field_name})
                feats_df = feats_df.merge(tmp_df, on='id', how='left')
        return feats_df                
    
    def make_feats(self, df, fillna=None):
        print("Starting to engineer features")
        data_df = df[['id', 'down_time', 'up_time']]
        merged_df = pd.DataFrame({'id': df['id'].unique().tolist()})
        data_df['time_diff'] = self.get_time_diff(data_df)
        
        for t_win in self.time_windows:
            win_field = f'window_{t_win}s'
            data_df[win_field] = self.get_time_windows(data_df, time_win=t_win*1000)

            print('Calculating window stats')
            feats_df = self.get_windows_stats(data_df, win_field)
            merged_df = merged_df.merge(feats_df, on='id', how='left')
        
        return merged_df

In [9]:
def update_activity(value):
    if value.startswith('Move'):
        return 'Move'
    if value.startswith('Remove/Cut'):
        return 'Remove'
    return value

In [10]:
class ShortcutsPreprocessor:
    def __init__(
        self, 
        train_df,
        column = 'down_event',
        vocab = [
            "control arrowleft", 
            "control arrowright", 
            "control c", 
            "control v", 
            "control x", 
            "control z",
            "control a",
            "leftclick", 
            "rightclick", 
            "middleclick", 
            "unknownclick"
        ],
        ngram_range = (1, 3),
        max_features = None
    ):
        self.train_df = train_df.copy()
        self.column = column
        self.vocab = vocab
        self.ngram_range = ngram_range
        self.token_pattern = r'(?u)\b\w+\b' # default=r”(?u)\b\w\w+\b”
        self.vectorizer = None
        self.initialized = False
        self.max_features = max_features
        
    def initialize(self):
        if self.initialized:
            return
        if self.column.__contains__('activity'):
            self.train_df['activity_changed'] = self.train_df['activity'].apply(update_activity)
            self.column = 'activity_changed'
            
        train_events_str_df = self.events_to_str(self.train_df)
        self.vectorizer = TfidfVectorizer(
                ngram_range=(1, 3), 
                token_pattern=self.token_pattern, 
                vocabulary=self.vocab
            )
        self.vectorizer.fit(train_events_str_df[self.column].to_numpy())
        self.train_df = None
        self.initialized = True
        
    def events_to_str(self, logs_df):
        events_str_df = logs_df.groupby(['id']).agg({self.column: list})[self.column].apply(
            lambda x: ' '.join(x)).to_frame().reset_index()
        return events_str_df
        
    def make_feats(self, logs_df):
        if not self.initialized:
            self.initialize()
            
        if self.column.__contains__('activity'):
            logs_df['activity_changed'] = logs_df['activity'].apply(update_activity)
            self.column = 'activity_changed'
            
        logs_events_str_df = self.events_to_str(logs_df)
        X_tokenized = self.vectorizer.transform(logs_events_str_df[self.column].to_numpy()).todense()
            
        feats_df = pd.DataFrame(
            X_tokenized, 
            index=logs_events_str_df['id'], 
            columns=[f'shortcuts_feature_{i}' for i in range(X_tokenized.shape[1])]
        )
#         feats_df.index.name = "id"
        return feats_df

In [11]:
def get_shortcuts_feats(logs_df, load_prepr=False, load_prepr_path="/kaggle/working"):
    if load_prepr:
        preprocessor_shortcuts = joblib.load(f"{load_prepr_path}/preprocessor_shortcuts")
    else:
        preprocessor_shortcuts = ShortcutsPreprocessor(logs_df)
        preprocessor_shortcuts.initialize()
        joblib.dump(preprocessor_shortcuts, f"preprocessor_shortcuts")

    shortcuts_feats = preprocessor_shortcuts.make_feats(logs_df).reset_index()
    return shortcuts_feats

In [12]:
def get_inner_word_pauses(x, min_len = 4):
    if x.shape[0] <= min_len:
        return np.NaN
    else:
        return x.iloc[1:].astype('float32').values

Functions get_iwp_stats and get_bwp_stats calculate inner word pauses and pauses before words or some events.

In [13]:
def get_iwp_stats(df):          
    df['up_time_lagged'] = df.groupby('id')['up_time'].shift(1).fillna(df['down_time'])
    df['time_diff'] = abs(df['down_time'] - df['up_time_lagged']) / 1000                          
    char_groups = df[df.down_event == 'q'].groupby(['id', 'word_count'])['time_diff']

    iw_pauses_sum = char_groups.apply(lambda x: np.sum(get_inner_word_pauses(x))).explode()
    iw_pauses_sum_mean = iw_pauses_sum.groupby('id').agg('mean')
    iw_pauses_sum_std = iw_pauses_sum.groupby('id').agg('std')
    iw_pauses_sum_entropy = iw_pauses_sum.groupby('id').agg(entropy_nan)
    iw_pauses_sum_first = iw_pauses_sum.groupby('id').agg('first')
    iw_pauses_sum_last = iw_pauses_sum.groupby('id').agg('last')
    
    result = pd.DataFrame({     
        'iw_pauses_sum_mean': iw_pauses_sum_mean,
        'iw_pauses_sum_std': iw_pauses_sum_std,
        'iw_pauses_sum_entropy': iw_pauses_sum_entropy,
        'iw_pauses_sum_first': iw_pauses_sum_first,
        'iw_pauses_sum_last': iw_pauses_sum_last,
    })
    return result

In [14]:
def get_before_word_pauses(x, min_len = 4):
    if x.shape[0] <= min_len:
        return np.NaN
    else:
        return x.iloc[0].astype('float32')

In [15]:
def get_bwp_stats(df, event='Leftclick'):          
    df['up_time_lagged'] = df.groupby('id')['up_time'].shift(1).fillna(df['down_time'])
#     df['up_time_lagged'] = df.groupby('id')['up_time'].shift(1).fillna(0)
    df['time_diff'] = abs(df['down_time'] - df['up_time_lagged']) / 1000                          
    char_groups = df[df.down_event == 'q'].groupby(['id', 'word_count'])['time_diff']
#     shift_chars = df[df.down_event == 'Shift'].groupby(['id'])['time_diff']

    bw_pauses_sum = char_groups.apply(lambda x: get_before_word_pauses(x)).explode()
    bw_pauses_sum_mean = bw_pauses_sum.groupby('id').agg('mean')
    bw_pauses_sum_std = bw_pauses_sum.groupby('id').agg('std')
    bw_pauses_sum_entropy = bw_pauses_sum.groupby('id').agg(entropy_nan)
    bw_pauses_sum_first = bw_pauses_sum.groupby('id').agg('first')
    bw_pauses_sum_last = bw_pauses_sum.groupby('id').agg('last')
    
    event_group = df[df.down_event == event].groupby(['id'])['time_diff']
    bep_sum = event_group.agg('sum')
    bep_mean = event_group.agg('mean')
    bep_std = event_group.agg('std')
#     bep_entropy = event_group.agg(entropy_nan)
#     bep_first = event_group.agg('first')
#     bep_last = event_group.agg('last')
#     bep_median = event_group.agg('median')
#     bep_max = event_group.agg('max')
#     bep_min = event_group.agg('min')
#     bep_iqr = event_group.agg(iqr_nan)
    
    result = pd.DataFrame({
        'bw_pauses_sum_mean': bw_pauses_sum_mean,
        'bw_pauses_sum_std': bw_pauses_sum_std,
        'bw_pauses_sum_entropy': bw_pauses_sum_entropy,
        'bw_pauses_sum_first': bw_pauses_sum_first,
        'bw_pauses_sum_last': bw_pauses_sum_last,
        
        'bep_sum': bep_sum,
        'bep_mean': bep_mean,
        'bep_std': bep_std,
#         'bep_entropy': bep_entropy,
#         'bep_first': bep_first,
#         'bep_last': bep_last,
#         'bep_max': bep_max,
#         'bep_min': bep_min,
#         'bep_median': bep_median,
#         'bep_iqr': bep_iqr,
    })
    result.fillna(0, inplace=True)
    return result

In [16]:
def get_tw_activities(logs_df, preprocessor):
    tw_feats = preprocessor.make_feats(logs_df, fillna=0)
    print("tw_feats.shape: ", tw_feats.shape)
    return tw_feats

Class RepetedGroupsPreprocessor finds bursts of some events and calculate statistics.

In [17]:
class RepetedGroupsPreprocessor:
    def __init__(self, column, word_to_find, aggregates = ['mean', 'std'], prefix='repeated_group_'):
        self.column = column
        self.word_to_find = word_to_find
        self.aggregates = aggregates
        self.prefix = prefix
        
    def get_matches(self, input_string):
        pattern = rf'\b{self.word_to_find}\b(?:\s+\b{self.word_to_find}\b)*'
        matches = re.findall(pattern, input_string)
        counts = [len(match.split()) for match in matches]
        return counts
        
    def make_feats(self, logs_df):
        events_str_df = logs_df.groupby(['id']).agg({self.column: list})[self.column].apply(
            lambda x: ' '.join(x)).to_frame()
        feats_df = events_str_df[self.column].apply(self.get_matches).explode().groupby('id').agg(self.aggregates)
        feats_df = feats_df.add_prefix(self.prefix).astype('float32').reset_index()
        feats_df.fillna(0, inplace=True)
        return feats_df

In [18]:
def get_tfidf_essay_feats(essay_df, load_prepr=False, load_prepr_path="/kaggle/working"):
    tfidf_feats_all = pd.DataFrame({'id': essay_df.id.unique()})
    token_pattern = r"(?u)\b[a-zA-Z][a-zA-Z]+\b|!|\?|\"|\'|\.|\,|\:\|\;|\-|\\|\/"
    vocab = ['qqqqqqqqq qqq', 'qqq qqqqqqqqqq', 'qqqqqqq .', 'qqq qq',
       'qqqq qqq', 'qqqqqq qqqqq', 'qq', 'qqqqq qqqq', 'qqqqqqq qqq',
       'qq qqqqq', 'qq qqqqqqqqq', 'qqqqqq ,', 'qqqqqq', 'qqq qqqqqqqq',
       'qqq qqqqqq', 'qqqqqqqqqq', 'qqqq qqqqqqq', 'qqqqqqq qqqqqqq',
       'qq qqq', 'qqqq .', ', qq', '. qqq', 'qqqqqq qqq', 'qqq qqqq', '.',
       'qqqq', 'qqqqqqqqqqq qqqq', 'qqqq qqqqqqqqq', 'qq qqqqqqq',
       'qqqqqqq ,', 'qqqq qqqqqqqqqqq', 'qqqqqq qqqqqqqqq',
       'qqqqqqqq qqqqq', 'qqqqqqqqqq qqq', 'qqqqqqqq qqq',
       'qqqqqq qqqqqqq', 'qqqqqqq', 'qqqqq qq', 'qqqqqqqq .', '. qqqqq',
       'qqqqq qqqqqqqq', 'qq .', 'qqqq qqqqq', 'qqqqqqqq ,',
       'qqqqqqqqqqq qq', '. qqqqqqq', 'qqqqq qqqqq', 'qqqqqqq qqqqq',
       '. qqqq', 'qqqqqqqqqqq .', ',', '!', '?', ':', ';', '-', '/', '\\']
    if load_prepr:
        vectorizer = joblib.load(f"{load_prepr_path}/vectorizer_essay")
    else:
        vectorizer = TfidfVectorizer(
            ngram_range=(1, 2), 
            token_pattern=token_pattern,
            vocabulary = vocab
        )
        vectorizer.fit(essay_df['essay'])
        joblib.dump(vectorizer, f"vectorizer_essay")
    X_tokenizer = vectorizer.transform(essay_df['essay']).todense()
    feats_vect_df = pd.DataFrame(
        X_tokenizer, 
        index=essay_df['id'], 
        columns=[f'tfidf_essay_feature_{i}' for i in range(X_tokenizer.shape[1])]
    )
    return feats_vect_df.reset_index()

Class VectorizedEventsPreprocessor calculates tf-idf statistics of events in columns.

In [19]:
class VectorizedEventsPreprocessor():
    def __init__(self, train_df, column, vocab, lowercase=False, token_pattern=r"[^\|]+", ngram_range=(1, 1)):
        self.train_df = train_df
        self.column = column
        self.vocab = vocab
        self.lowercase = lowercase
        self.token_pattern = token_pattern
        self.ngram_range = ngram_range
        self.initialized = False
        self.vectorizer = None
        
    def initialize(self):
        if self.initialized:
            return
        train_events_str_df = self.events_to_str(self.train_df)
        self.vectorizer = TfidfVectorizer(
            ngram_range=self.ngram_range, 
            vocabulary=self.vocab, 
            lowercase=self.lowercase, 
            token_pattern=self.token_pattern
        )
        self.vectorizer.fit(train_events_str_df[self.column].to_numpy())
        self.train_df = None
        self.initialized = True
        
    def events_to_str(self, logs_df):
        events_str_df = logs_df.groupby(['id']).agg({self.column: list})[self.column].apply(
            lambda x: '|'.join(x)).to_frame().reset_index()
        return events_str_df
        
    def make_feats(self, logs_df):
        if not self.initialized:
            self.initialize()
        logs_events_str_df = self.events_to_str(logs_df)
        X_tokenizer = self.vectorizer.transform(logs_events_str_df[self.column].to_numpy()).todense()
        feats_vect_df = pd.DataFrame(
            X_tokenizer, 
            index=logs_events_str_df['id'], 
            columns=[f'tfidf_{self.column}_feature_{i}' for i in range(X_tokenizer.shape[1])]
        )
        return feats_vect_df

In [20]:
def get_tfidf_feats(logs_df, load_prepr=False, load_prepr_path="/kaggle/working"):
    activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
    events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
    text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
    fields_values_dict = {
        'activity': activities,
        'down_event': events,
        'up_event': events,
        'text_change': text_changes,
    }
    tfidf_feats_all = pd.DataFrame({'id': logs_df.id.unique()})
    
    for field, values in fields_values_dict.items():
        print(f" TfIdf, field: {field}")
        if load_prepr:
            preprocessor_tfidf = joblib.load(f"{load_prepr_path}/preprocessor_tfidf_{field}")
        else:
            preprocessor_tfidf = VectorizedEventsPreprocessor(logs_df, field, values)
            preprocessor_tfidf.initialize()
            joblib.dump(preprocessor_tfidf, f"preprocessor_tfidf_{field}")

        tfidf_feats = preprocessor_tfidf.make_feats(logs_df).reset_index()
        tfidf_feats_all = tfidf_feats_all.merge(tfidf_feats, on='id', how='left')
    return tfidf_feats_all

# Extract features

In [21]:
agg_methods = ['mean', 'std', entropy_nan]

In [22]:
preprocessor_tw_a_1 = PreprocessorTimeWindows_Activities(
    agg_methods, 
    time_windows=[60], 
    filter_field='activity', 
    filter_values=['Input'],
    field_suffix='_inp',
)

In [23]:
preprocessor_tw_a_2 = PreprocessorTimeWindows_Activities(
    agg_methods, 
    time_windows=[60], 
    filter_field='activity', 
    filter_values=['Remove/Cut'],
    field_suffix='_rem_all',
)

In [24]:
# preprocessor_tw_a_3 = PreprocessorTimeWindows_Activities(
#     agg_methods, 
#     time_windows=[60], 
#     filter_field='down_event', 
#     filter_values=['Backspace'], 
#     field_suffix='_bsp'
# )

In [25]:
# preprocessor_tw_a_4 = PreprocessorTimeWindows_Activities(
#     agg_methods, 
#     time_windows=[60], 
#     filter_field='down_event', 
#     filter_values=['q'], 
#     field_suffix='_q'
# )

In [26]:
preprocessor_tw_p = PreprocessorTimeWindows_Pauses(
    agg_methods,
    time_windows=[60]
)

In [27]:
preprocessor_rep_groups_1 = RepetedGroupsPreprocessor(
    column='activity', 
    word_to_find='Remove/Cut',
    aggregates = agg_methods,
    prefix='repeated_remove_'
)

In [28]:
preprocessor_rep_groups_2 = RepetedGroupsPreprocessor(
    column='down_event', 
    word_to_find='q', 
    aggregates = agg_methods,
    prefix='repeated_q_'
)

In [29]:
def get_feats_all(logs_file_name='train_logs.csv', load_prepr_path="/kaggle/working"):
    print('get_feats_all')
    logs_df = pd.read_csv(CONFIG.data_path + logs_file_name)
    essays_df = get_essay_df(logs_df)

    feats_df = logs_df.groupby('id')['event_id'].agg('max').to_frame().add_suffix("_max").reset_index()
    feats_df = feats_df.merge(
        logs_df.groupby('id')['cursor_position'].agg(relextrema_count).to_frame().add_suffix("_relextrema").reset_index(), 
        on='id', how='left'
    )
    
    print("< Get Essay tfidf stats >")
    load_prepr = logs_file_name.__contains__("test")
    feats_df = feats_df.merge(
        get_tfidf_essay_feats(essays_df, load_prepr=load_prepr, load_prepr_path=load_prepr_path), 
        on='id', how='left'
    )
    
    print("< Get Logs tfidf stats >")
    feats_df = feats_df.merge(get_tfidf_feats(logs_df), on='id', how='left')
    
#     print("< Get shortcuts stats >")
#     feats_df = feats_df.merge(get_shortcuts_feats(logs_df), on='id', how='left')
    
    print("< get bwp stats >")
    feats_df = feats_df.merge(get_bwp_stats(logs_df), on='id', how='left')
    
    print("< tw_activities Preprocessor 1 >")
    tw_activities_feats = preprocessor_tw_a_1.make_feats(logs_df, fillna=0)
    feats_df = feats_df.merge(tw_activities_feats, on='id', how='left')

    print("< tw_activities Preprocessor 2 >")
    tw_activities_feats = preprocessor_tw_a_2.make_feats(logs_df, fillna=0)
    feats_df = feats_df.merge(tw_activities_feats, on='id', how='left')
    
    print("< tw_pauses Preprocessor >")
    tw_activities_feats = preprocessor_tw_p.make_feats(logs_df, fillna=0)
    feats_df = feats_df.merge(tw_activities_feats, on='id', how='left')
    
    print("< Get RepetedGroups stats >")
    feats_rep_gr_df = preprocessor_rep_groups_1.make_feats(logs_df)
    feats_df = feats_df.merge(feats_rep_gr_df, on='id', how='left')
    
    feats_rep_gr_df = preprocessor_rep_groups_2.make_feats(logs_df)
    feats_df = feats_df.merge(feats_rep_gr_df, on='id', how='left')
    
    return feats_df

In [30]:
if __name__ == '__main__':
    train_feats = get_feats_all('train_logs.csv')
    print()
    display(train_feats)

get_feats_all
< Get Essay tfidf stats >
< Get Logs tfidf stats >
 TfIdf, field: activity
 TfIdf, field: down_event
 TfIdf, field: up_event
 TfIdf, field: text_change
< get bwp stats >
< tw_activities Preprocessor 1 >
Starting to engineer features
Calculating window stats


100%|██████████| 3/3 [01:40<00:00, 33.58s/it, method=entropy_nan]


< tw_activities Preprocessor 2 >
Starting to engineer features
Calculating window stats


100%|██████████| 3/3 [01:29<00:00, 29.96s/it, method=entropy_nan]


< tw_pauses Preprocessor >
Starting to engineer features
Calculating window stats


100%|██████████| 3/3 [04:19<00:00, 86.61s/it, method=entropy_nan, pause=pause_2_sec]


< Get RepetedGroups stats >



,id,event_id_max,cursor_position_relextrema,tfidf_essay_feature_0,tfidf_essay_feature_1,tfidf_essay_feature_2,tfidf_essay_feature_3,tfidf_essay_feature_4,tfidf_essay_feature_5,tfidf_essay_feature_6,tfidf_essay_feature_7,tfidf_essay_feature_8,tfidf_essay_feature_9,tfidf_essay_feature_10,tfidf_essay_feature_11,tfidf_essay_feature_12,tfidf_essay_feature_13,tfidf_essay_feature_14,tfidf_essay_feature_15,tfidf_essay_feature_16,tfidf_essay_feature_17,tfidf_essay_feature_18,tfidf_essay_feature_19,tfidf_essay_feature_20,tfidf_essay_feature_21,tfidf_essay_feature_22,tfidf_essay_feature_23,tfidf_essay_feature_24,tfidf_essay_feature_25,tfidf_essay_feature_26,tfidf_essay_feature_27,tfidf_essay_feature_28,tfidf_essay_feature_29,tfidf_essay_feature_30,tfidf_essay_feature_31,tfidf_essay_feature_32,tfidf_essay_feature_33,tfidf_essay_feature_34,tfidf_essay_feature_35,tfidf_essay_feature_36,tfidf_essay_feature_37,tfidf_essay_feature_38,tfidf_essay_feature_39,tfidf_essay_feature_40,tfidf_essay_feature_41,tfidf_essay_feature_42,tfidf_essay_feature_43,tfidf_essay_feature_44,tfidf_essay_feature_45,tfidf_essay_feature_46,tfidf_essay_feature_47,tfidf_essay_feature_48,tfidf_essay_feature_49,tfidf_essay_feature_50,tfidf_essay_feature_51,tfidf_essay_feature_52,tfidf_essay_feature_53,tfidf_essay_feature_54,tfidf_essay_feature_55,tfidf_essay_feature_56,tfidf_essay_feature_57,tfidf_activity_feature_0,tfidf_activity_feature_1,tfidf_activity_feature_2,tfidf_activity_feature_3,tfidf_activity_feature_4,tfidf_down_event_feature_0,tfidf_down_event_feature_1,tfidf_down_event_feature_2,tfidf_down_event_feature_3,tfidf_down_event_feature_4,tfidf_down_event_feature_5,tfidf_down_event_feature_6,tfidf_down_event_feature_7,tfidf_down_event_feature_8,tfidf_down_event_feature_9,tfidf_down_event_feature_10,tfidf_down_event_feature_11,tfidf_down_event_feature_12,tfidf_down_event_feature_13,tfidf_down_event_feature_14,tfidf_down_event_feature_15,tfidf_up_event_feature_0,tfidf_up_event_feature_1,tfidf_up_event_feature_2,tfidf_up_event_feature_3,tfidf_up_event_feature_4,tfidf_up_event_feature_5,tfidf_up_event_feature_6,tfidf_up_event_feature_7,tfidf_up_event_feature_8,tfidf_up_event_feature_9,tfidf_up_event_feature_10,tfidf_up_event_feature_11,tfidf_up_event_feature_12,tfidf_up_event_feature_13,tfidf_up_event_feature_14,tfidf_up_event_feature_15,tfidf_text_change_feature_0,tfidf_text_change_feature_1,tfidf_text_change_feature_2,tfidf_text_change_feature_3,tfidf_text_change_feature_4,tfidf_text_change_feature_5,tfidf_text_change_feature_6,tfidf_text_change_feature_7,tfidf_text_change_feature_8,tfidf_text_change_feature_9,tfidf_text_change_feature_10,tfidf_text_change_feature_11,tfidf_text_change_feature_12,tfidf_text_change_feature_13,bw_pauses_sum_mean,bw_pauses_sum_std,bw_pauses_sum_entropy,bw_pauses_sum_first,bw_pauses_sum_last,bep_sum,bep_mean,bep_std,window_60s_activity_count_mean_inp,window_60s_activity_count_std_inp,window_60s_activity_count_entropy_nan_inp,window_60s_activity_count_mean_rem_all,window_60s_activity_count_std_rem_all,window_60s_activity_count_entropy_nan_rem_all,window_60s_pause_1_sec_mean,window_60s_pause_1_sec_std,window_60s_pause_1_sec_entropy_nan,window_60s_pause_15_sec_mean,window_60s_pause_15_sec_std,window_60s_pause_15_sec_entropy_nan,window_60s_pause_2_sec_mean,window_60s_pause_2_sec_std,window_60s_pause_2_sec_entropy_nan,repeated_remove_mean,repeated_remove_std,repeated_remove_entropy_nan,repeated_q_mean,repeated_q_std,repeated_q_entropy_nan
0,001519c8,2557,237,0.030876,0.033073,0.030573,0.039390,0.090986,0.043700,0.687499,0.026207,0.041034,0.078621,0.058290,0.034905,0.272405,0.041502,0.065438,0.105168,0.013606,0.073942,0.155723,0.042078,0.016433,0.014204,0.052563,0.025975,0.181604,0.466981,0.000000,0.043986,0.013317,0.017137,0.000000,0.000000,0.017305,0.000000,0.000000,0.047729,0.246462,0.130401,0.000000,0.000000,0.018051,0.000000,0.078526,0.020245,0.052224,0.000000,0.085085,0.031169,0.041485,0.021090,0.137168,0.0,0.000000,0.0,0.0,0.000000,0.0,0

In [31]:
if __name__ == '__main__':
    train_feats.to_csv('train_feats_extra.csv', index=False)

In [32]:
if __name__ == '__main__':
    test_feats = get_feats_all('test_logs.csv')
    print()
    display(test_feats)

get_feats_all
< Get Essay tfidf stats >
< Get Logs tfidf stats >
 TfIdf, field: activity
 TfIdf, field: down_event
 TfIdf, field: up_event
 TfIdf, field: text_change
< get bwp stats >
< tw_activities Preprocessor 1 >
Starting to engineer features
Calculating window stats


100%|██████████| 3/3 [00:00<00:00, 66.20it/s, method=entropy_nan]


< tw_activities Preprocessor 2 >
Starting to engineer features
Calculating window stats


100%|██████████| 3/3 [00:00<00:00, 97.31it/s, method=entropy_nan]


< tw_pauses Preprocessor >
Starting to engineer features
Calculating window stats


100%|██████████| 3/3 [00:00<00:00, 23.19it/s, method=entropy_nan, pause=pause_2_sec]


< Get RepetedGroups stats >



,id,event_id_max,cursor_position_relextrema,tfidf_essay_feature_0,tfidf_essay_feature_1,tfidf_essay_feature_2,tfidf_essay_feature_3,tfidf_essay_feature_4,tfidf_essay_feature_5,tfidf_essay_feature_6,tfidf_essay_feature_7,tfidf_essay_feature_8,tfidf_essay_feature_9,tfidf_essay_feature_10,tfidf_essay_feature_11,tfidf_essay_feature_12,tfidf_essay_feature_13,tfidf_essay_feature_14,tfidf_essay_feature_15,tfidf_essay_feature_16,tfidf_essay_feature_17,tfidf_essay_feature_18,tfidf_essay_feature_19,tfidf_essay_feature_20,tfidf_essay_feature_21,tfidf_essay_feature_22,tfidf_essay_feature_23,tfidf_essay_feature_24,tfidf_essay_feature_25,tfidf_essay_feature_26,tfidf_essay_feature_27,tfidf_essay_feature_28,tfidf_essay_feature_29,tfidf_essay_feature_30,tfidf_essay_feature_31,tfidf_essay_feature_32,tfidf_essay_feature_33,tfidf_essay_feature_34,tfidf_essay_feature_35,tfidf_essay_feature_36,tfidf_essay_feature_37,tfidf_essay_feature_38,tfidf_essay_feature_39,tfidf_essay_feature_40,tfidf_essay_feature_41,tfidf_essay_feature_42,tfidf_essay_feature_43,tfidf_essay_feature_44,tfidf_essay_feature_45,tfidf_essay_feature_46,tfidf_essay_feature_47,tfidf_essay_feature_48,tfidf_essay_feature_49,tfidf_essay_feature_50,tfidf_essay_feature_51,tfidf_essay_feature_52,tfidf_essay_feature_53,tfidf_essay_feature_54,tfidf_essay_feature_55,tfidf_essay_feature_56,tfidf_essay_feature_57,tfidf_activity_feature_0,tfidf_activity_feature_1,tfidf_activity_feature_2,tfidf_activity_feature_3,tfidf_activity_feature_4,tfidf_down_event_feature_0,tfidf_down_event_feature_1,tfidf_down_event_feature_2,tfidf_down_event_feature_3,tfidf_down_event_feature_4,tfidf_down_event_feature_5,tfidf_down_event_feature_6,tfidf_down_event_feature_7,tfidf_down_event_feature_8,tfidf_down_event_feature_9,tfidf_down_event_feature_10,tfidf_down_event_feature_11,tfidf_down_event_feature_12,tfidf_down_event_feature_13,tfidf_down_event_feature_14,tfidf_down_event_feature_15,tfidf_up_event_feature_0,tfidf_up_event_feature_1,tfidf_up_event_feature_2,tfidf_up_event_feature_3,tfidf_up_event_feature_4,tfidf_up_event_feature_5,tfidf_up_event_feature_6,tfidf_up_event_feature_7,tfidf_up_event_feature_8,tfidf_up_event_feature_9,tfidf_up_event_feature_10,tfidf_up_event_feature_11,tfidf_up_event_feature_12,tfidf_up_event_feature_13,tfidf_up_event_feature_14,tfidf_up_event_feature_15,tfidf_text_change_feature_0,tfidf_text_change_feature_1,tfidf_text_change_feature_2,tfidf_text_change_feature_3,tfidf_text_change_feature_4,tfidf_text_change_feature_5,tfidf_text_change_feature_6,tfidf_text_change_feature_7,tfidf_text_change_feature_8,tfidf_text_change_feature_9,tfidf_text_change_feature_10,tfidf_text_change_feature_11,tfidf_text_change_feature_12,tfidf_text_change_feature_13,bw_pauses_sum_mean,bw_pauses_sum_std,bw_pauses_sum_entropy,bw_pauses_sum_first,bw_pauses_sum_last,bep_sum,bep_mean,bep_std,window_60s_activity_count_mean_inp,window_60s_activity_count_std_inp,window_60s_activity_count_entropy_nan_inp,window_60s_activity_count_mean_rem_all,window_60s_activity_count_std_rem_all,window_60s_activity_count_entropy_nan_rem_all,window_60s_pause_1_sec_mean,window_60s_pause_1_sec_std,window_60s_pause_1_sec_entropy_nan,window_60s_pause_15_sec_mean,window_60s_pause_15_sec_std,window_60s_pause_15_sec_entropy_nan,window_60s_pause_2_sec_mean,window_60s_pause_2_sec_std,window_60s_pause_2_sec_entropy_nan,repeated_remove_mean,repeated_remove_std,repeated_remove_entropy_nan,repeated_q_mean,repeated_q_std,repeated_q_entropy_nan
0,0000aaaa,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0

In [33]:
if __name__ == '__main__':
    drop_features = ['id']
    train_scores = pd.read_csv(CONFIG.data_path + 'train_scores.csv')
    data = train_feats.merge(train_scores, on='id', how='left')
    x = data.drop(drop_features + ['score'], axis=1)
    y = data['score'].values
    print(f'Number of features: {len(x.columns)}')

Number of features: 140


In [34]:
# train_logs_df = pd.read_csv(CONFIG.data_path + "train_logs.csv")
# test_logs_df = pd.read_csv(CONFIG.data_path + 'test_logs.csv')

In [35]:
# train_bwp_df = get_bwp_stats(train_logs_df)
# test_bwp_df = get_bwp_stats(test_logs_df)

In [36]:
# bep_new_cols = ['bep_last', 'bep_max', 'bep_median', 'bep_iqr']

In [37]:
# train_bwp_df[bep_new_cols]

In [38]:
# train_feats = train_feats.merge(train_bwp_df[bep_new_cols], how='left', on='id')
# test_feats = test_feats.merge(test_bwp_df[bep_new_cols], how='left', on='id')

In [39]:
# train_feats

# Training and Evaluation

In [40]:
def train_valid_split(data_x, data_y, train_idx, valid_idx):
    x_train = data_x.iloc[train_idx]
    y_train = data_y[train_idx]
    x_valid = data_x.iloc[valid_idx]
    y_valid = data_y[valid_idx]
    return x_train, y_train, x_valid, y_valid

In [41]:
def evaluate(data_x, data_y, model, random_state=42, n_splits=5):
    all_models = []
    skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    oof_pred = np.zeros(len(data_x))
    for i, (train_index, valid_index) in enumerate(skf.split(data_x, data_y.astype(str))):
        train_x, train_y, valid_x, valid_y = train_valid_split(data_x, data_y, train_index, valid_index)
        model.fit(
            train_x, 
            train_y,
#             eval_metric='rmse',
#             eval_set=[(valid_x, valid_y)],
#             early_stopping_rounds=50,
#             verbose=1, 
        )
        all_models.append(deepcopy(model))
        oof_pred[valid_index] = model.predict(valid_x)
    return oof_pred, all_models

In [42]:
if __name__ == '__main__' and CONFIG.solution_name == 'XGBRegressor':
    param = {
        'n_estimators': 728,
        'max_depth': 5,
        'reg_alpha': 0.7304170274014448,
        'reg_lambda': 2.4242433866950845,
        'subsample': 0.3680380596141958,
        'colsample_bynode': 0.1495284704870805,
        'eta': 0.014041211054774062,
        'eval_metric': 'rmse',
        'random_state': 41
    }
    solution = XGBRegressor(**param)

In [43]:
if __name__ == '__main__':
    test_ids = test_feats['id'].values
    testin_x = test_feats.drop(drop_features, axis=1)

    print('< Learning and Evaluation >')
    oof_pred, all_models = evaluate(
        x.copy(),
        y.copy(), 
        solution, 
        random_state=CONFIG.random_state,
        n_splits=CONFIG.n_splits
    )

< Learning and Evaluation >


In [44]:
if __name__ == '__main__':
    rmse = mean_squared_error(y, oof_pred, squared=False)
    print(f"OOF RMSE: {rmse}")

OOF RMSE: 0.6158370116917163


In [45]:
if __name__ == '__main__':
    voting_reg = VotingRegressor(estimators=all_models)
    voting_reg.estimators_ = all_models
    voting_reg_preds = voting_reg.predict(testin_x.copy())

In [46]:
if __name__ == '__main__':
    joblib.dump(voting_reg, "extra_features.model")
    joblib.dump(oof_pred, "extra_features_oof_preds")

# Submission

In [47]:
if __name__ == '__main__':
    submission_df = pd.DataFrame({
        'id': test_ids, 
        'score': voting_reg_preds
    })
    submission_df.to_csv('submission.csv', index=False)
    display(submission_df)

,id,score
0,0000aaaa,1.833074
1,2222bbbb,1.975253
2,4444cccc,1.953716
